<a href="https://colab.research.google.com/github/scarlettogrady/train_dataset/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import difflib
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import pandas as pd
from scipy import stats

# **Data Import**

In [171]:
!curl -LJO https://raw.githubusercontent.com/scarlettogrady/train_dataset/main/train.csv
src1 = 'train.csv'

!curl -LJO https://raw.githubusercontent.com/scarlettogrady/train_dataset/main/test.csv
src2 = 'test.csv'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1375k  100 1375k    0     0  2675k      0 --:--:-- --:--:-- --:--:-- 2675k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  331k  100  331k    0     0  1397k      0 --:--:-- --:--:-- --:--:-- 1404k


# **Pre-processing**

In [172]:
import pandas as pd
from scipy import stats

#from sklearn.linear_model import LinearRegression
#from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

def load(csv_file):
  dataset_original = pd.read_csv(csv_file)
  df = pd.DataFrame(dataset_original)
  return df


df = pd.read_csv(src1)
#df = pd.read_csv("/content/drive/MyDrive/train.csv")
df_test = load(src2)
feature_types = df.dtypes

#print(feature_types)


# remove rows with incomplete data
df_cleaned = df.dropna()

#missing_values = df_cleaned.isnull().sum()

#print(missing_values)
# Convert relevant columns to numerical data types
numeric_columns = ['Tank Failure Pressure (bar)', 'Liquid Ratio (%)', 'Tank Width (m)',
                   'Tank Length (m)', 'Tank Height (m)', 'BLEVE Height (m)',
                   'Vapour Height (m)', 'Vapour Temperature (K)', 'Liquid Temperature (K)',
                   'Obstacle Distance to BLEVE (m)', 'Obstacle Width (m)', 'Obstacle Height (m)',
                   'Obstacle Thickness (m)', 'Obstacle Angle', 'Liquid Critical Pressure (bar)',
                   'Liquid Boiling Temperature (K)', 'Liquid Critical Temperature (K)',
                   'Sensor ID', 'Sensor Position x', 'Sensor Position y',
                   'Sensor Position z', 'Target Pressure (bar)']

#df_cleaned[numeric_columns] = df_cleaned[numeric_columns].apply(pd.to_numeric, errors='coerce')
df_cleaned.loc[:, numeric_columns] = df_cleaned.loc[:, numeric_columns].apply(pd.to_numeric, errors='coerce')


# Calculate Q1, Q3, and IQR for each feature
Q1 = df_cleaned[numeric_columns].quantile(0.25)
Q3 = df_cleaned[numeric_columns].quantile(0.75)
IQR = Q3 - Q1

# Define the outlier threshold
#threshold = 1.5
threshold = 1.5

# Identify outliers based on the IQR method
outliers = ((df_cleaned[numeric_columns] < (Q1 - threshold * IQR)) | (df_cleaned[numeric_columns] > (Q3 + threshold * IQR))).any(axis=1)

# removed incomplete data and outliers
df_cleaned2 = df_cleaned[~outliers]

# Print the outliers
#print(df_cleaned[outliers])

# 1068 outliers detected
#num_outliers = outliers.sum()
#print(f"Number of rows with outliers: {num_outliers}")

#num_rows1 = df_cleaned.shape[0]
#num_rows2 = df_cleaned2.shape[0]

#print("Total number of rows in cleaned:", num_rows1)
#print("Total number of rows in cleaned2:", num_rows2)

#num_duplicates = df_cleaned2.duplicated().sum()

#print("number of duplicated rows:", num_duplicates)

df_cleaned3 = df_cleaned2.drop_duplicates()



# Dot notation (if the column name doesn't contain spaces or special characters)
#print(df_test.column_name.head(10))
status_mapping = {
    'superheated': 'Superheated',
    'Superheated ': 'Superheated',
    'superheated ': 'Superheated',
    'subcooled': 'Subcooled',
    'Subcooled ': 'Subcooled',
    'subcooled ': 'Subcooled',
    'Subcool': 'Subcooled',
    'Subcoled': 'Subcooled',
    'Superheat': 'Superheated',
    'Saperheated': 'Superheated'
}

df_cleaned3 = df_cleaned3.dropna()

# Replace the incorrect spellings with the correct values
df_cleaned3['Status'] = df_cleaned3['Status'].replace(status_mapping)

# Replace remaining missing or invalid values with 'Unknown'
#df_train['Status'] = df_train['Status'].fillna('Unknown')

# Map 'Superheated' to 1 and 'Subcooled' to 0
status_mapping_binary = {
    'Superheated': 1,
    'Subcooled': 0
}

df_cleaned3['Status'] = df_cleaned3['Status'].map(status_mapping_binary)
df_test['Status'] = df_test['Status'].map(status_mapping_binary)

def normalisation(df):
  scaler = MinMaxScaler()
  target = None
  if 'Target Pressure (bar)' in df.columns:
        target = df['Target Pressure (bar)']
        df = df.drop(columns=['Target Pressure (bar)'])
        target = target.reset_index(drop=True)
  new_df = df.drop(columns = ['ID', 'Status'])
  normalised_data = scaler.fit_transform(new_df)
  normalised_df = pd.DataFrame(normalised_data, columns=new_df.columns)
  df_reset = df.reset_index(drop=True)
  normalised_df.insert(0, 'ID', df_reset['ID'] )
  normalised_df.insert(11, 'Status', df_reset['Status'])
  if target is not None:
    normalised_df.insert(normalised_df.shape[1], 'Target Pressure (bar)', target)

  return normalised_df

df_cleaned3 = normalisation(df_cleaned3)
df_test = normalisation(df_test)

# **Model 1 - Linear Regression**

In [ ]:
def LinearRegressionTrain(df_trainset):
    features_train = df_trainset.drop(columns=['Target Pressure (bar)', 'ID'])
    target_train = df_trainset['Target Pressure (bar)']

    # Splitting the training data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

    # Creating an instance of Linear Regression model
    model = LinearRegression()

    # Fitting the model to the training data
    model.fit(X_train, y_train)

    # Making predictions on the validation set
    y_pred = model.predict(X_val)

    # Evaluating the model
    mse = mean_squared_error(y_val, y_pred)
    print("Mean Squared Error on Validation Set:", mse)

    mape = mean_absolute_percentage_error(y_val, y_pred)
    print("Mean Absolute Percentage Error on Validation Set:", mape)

    return model

def LinearRegressionTest(test_dataset, LRModel):
    features_test = test_dataset.drop(columns=['ID'])  # Exclude 'ID' from features
    test_predictions = model.predict(features_test)

    # Creating a DataFrame for the predictions with 'ID' as the first column and predicted 'Target Pressure' as the second column
    df_test_predictions = pd.DataFrame({'ID': test_dataset['ID'], 'Predicted Target Pressure (bar)': test_predictions})
    # Saving the predictions to a CSV file
    df_test_predictions.to_csv('LR_test_predictions.csv', index=False)

    return df_test_predictions

**Testing Model 1**

In [ ]:
"""df_train = preProcessingTrain(src1)

df_test = preProcessingTrain(src2)

df_train.iloc[:, 9:20]"""

model = LinearRegressionTrain(df_cleaned3)
df_test_predictions = LinearRegressionTest(df_test, model)

#df_test_predictions

Mean Squared Error on Validation Set: 0.014650181356989728
Mean Absolute Percentage Error on Validation Set: 0.5619472182226393


# **Model 2 - Random Forest Trees**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

def RandomTreesModel(train_dataset):
    features_train = train_dataset.drop(columns=['Target Pressure (bar)', 'ID'])
    target_train = train_dataset['Target Pressure (bar)']

    # Splitting the training data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

    # Define the hyperparameter grid
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
    }

    # Create the base model to tune
    rf_regressor = RandomForestRegressor(random_state=42)

    # Random search of parameters, using 3 fold cross validation,
    # search across 100 different combinations, and use all available cores
    random_search = RandomizedSearchCV(estimator = rf_regressor, param_distributions = param_grid, n_iter = 30, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    random_search.fit(X_train, y_train)

    # Get the best model
    best_rf_regressor = random_search.best_estimator_

    # Make predictions on the validation set
    y_pred = best_rf_regressor.predict(X_val)

    # Calculate evaluation metrics
    mse = mean_squared_error(y_val, y_pred)
    print("Mean Squared Error on Validation Set:", mse)

    mape = mean_absolute_percentage_error(y_val, y_pred)
    print("Mean Absolute Percentage Error on Validation Set:", mape)

    return best_rf_regressor


def test_model(test_dataset, model):
    features_test = test_dataset.drop(columns=['ID'])

    # Make predictions on the test set
    y_pred = model.predict(features_test)

    # Creating a DataFrame for the predictions with 'ID' as the first column and predicted 'Target Pressure' as the second column
    df_test_predictions = pd.DataFrame({'ID': test_dataset['ID'], 'Predicted Target Pressure (bar)': y_pred})
    # Saving the predictions to a CSV file
    df_test_predictions.to_csv('test_predictions.csv', index=False)

    return df_test_predictions


**Testing Model 2**

In [ ]:
# Assuming regressor is the trained Random Forest regressor model
#df_train = preProcessingTrain(src1)
#df_test = preProcessingTrain(src2)

regressor = RandomTreesModel(df_cleaned3)
df_test_pred = test_model(df_test, regressor)
df_test_pred

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Mean Squared Error on Validation Set: 0.0031792088746636425
Mean Absolute Percentage Error on Validation Set: 0.17972307840736718


,ID,Predicted Target Pressure (bar)
0,0,0.270208
1,1,0.272481
2,2,0.337013
3,3,0.321587
4,4,0.292605
...,...,...
3198,3198,0.568911
3199,3199,0.485128
3200,3200,0.400777
3201,3201,0.455843


# **Model 3 - Neural Networks**

In [169]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

def NeuralMLPNetwork(train_dataset, best_params=None):
    features_train = train_dataset.drop(columns=['Target Pressure (bar)', 'ID'])
    target_train = train_dataset['Target Pressure (bar)']

    # Splitting the training data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

    if best_params is None:
        # If best_params is not provided, perform grid search to find the best hyperparameters
        mlp_regressor = MLPRegressor(random_state=42)

        param_grid = {
            'hidden_layer_sizes': [(200, 150, 100, 50), (250, 200, 150, 100), (300, 200, 100)],
            'alpha': [0.0001, 0.001, 0.01],
            'learning_rate_init': [0.001, 0.01, 0.1]
        }
        grid_search = GridSearchCV(mlp_regressor, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        print("Best NN Hyperparameters:", grid_search.best_params_)
        best_model = grid_search.best_estimator_
    else:
        # Use the provided hyperparameters
        best_model = MLPRegressor(**best_params)

    best_model.fit(X_train, y_train)

    y_pred_val = best_model.predict(X_val)

    # Calculate Mean Absolute Error on Validation Set
    mae = mean_absolute_percentage_error(y_val, y_pred_val)
    print("Mean Absolute Percentage Error on Validation Set:", mae)

    return best_model

def NN_test_model(test_dataset, model):
    features_test = test_dataset.drop(columns=['ID'])

    # Make predictions on the test set
    y_pred_test = model.predict(features_test)

    # Creating a DataFrame for the predictions with 'ID' as the first column and predicted 'Target Pressure' as the second column
    df_test_predictions = pd.DataFrame({'ID': test_dataset['ID'], 'Predicted Target Pressure (bar)': y_pred_test})
    # Saving the predictions to a CSV file
    df_test_predictions.to_csv('NN_test_predictions.csv', index=False)

    return df_test_predictions


**Testing Model 3**

In [173]:
best_hyperparameters = {'alpha': 0.01, 'hidden_layer_sizes': (250, 200, 150, 100), 'learning_rate_init': 0.001}

NNMLP = NeuralMLPNetwork(df_cleaned3, best_params=best_hyperparameters)
#NNMLP = NeuralMLPNetwork(df_cleaned3)
df_NN_test_pred = NN_test_model(df_test, NNMLP)
df_NN_test_pred

Mean Absolute Percentage Error on Validation Set: 0.1517991437949294


,ID,Predicted Target Pressure (bar)
0,0,0.322467
1,1,0.357040
2,2,0.390674
3,3,0.295407
4,4,0.340144
...,...,...
3198,3198,0.605391
3199,3199,0.453807
3200,3200,0.311309
3201,3201,0.363845
